In [5]:
#import librarys for data analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from speedml import Speedml
from ydata_profiling import ProfileReport

%matplotlib inline

In [12]:
#read in cleaned landslide data
clean_slide=pd.read_parquet("../1_data/CleanedLS.parquet")
clean_slide

,landslide_id,landslide_date,located_accuracy,category_landslide,landslide_trig,landslide_area,landslide_env,injuries,fatalities,risk
7209,10393,2017-09-28,exact,rock_fall,no_apparent_trigger,large,natural_slope,1,0,True
7210,10392,2017-09-27,exact,rock_fall,no_apparent_trigger,medium,natural_slope,1,1,True
3523,11013,2017-09-26,5km,mudslide,downpour,medium,natural_slope,0,0,False
3945,10971,2017-09-11,25km,landslide,flooding,unknown,unknown,0,2,True
5174,10991,2017-09-10,25km,landslide,rain,medium,natural_slope,9,0,True
...,...,...,...,...,...,...,...,...,...,...
9540,7831,1996-12-26,50km,debris_flow,freeze_thaw,small,above_river,0,0,False
9484,7830,1996-12-26,50km,debris_flow,freeze_thaw,small,above_road,0,0,False
9838,7408,1995-08-14,10km,debris_flow,downpour,large,unknown,0,0,False
10768,7331,1993-06-04,1km,other,rain,medium,deforested_slope,0,0,False


2023-04-10 19:31:30,566 - INFO     - Executing shutdown due to inactivity...
2023-04-10 19:31:46,998 - INFO     - Executing shutdown...
2023-04-10 19:31:47,000 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "C:\Users\shupe\anaconda3\envs\data_science\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\shupe\anaconda3\envs\data_science\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\shupe\anaconda3\envs\data_science\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\shupe\anaconda3\envs\data_science\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\shupe\anaconda3\envs\data_science\lib\site-packages\dtale\app.py", line 435, in shutdown
  

In [11]:
#create quick data analysis summary
Landslide_basics=ProfileReport(clean_slide,title="Landslide Quick Analysis",dark_mode=True)
clean_slide.describe()

,landslide_id,injuries,fatalities
count,10944.000000,10944.000000,10944.000000
mean,5559.200475,0.362208,2.825567
std,3231.955714,5.928803,56.237670
min,1.000000,0.000000,0.000000
25%,2762.750000,0.000000,0.000000
50%,5518.500000,0.000000,0.000000
75%,8367.250000,0.000000,0.000000
max,11221.000000,374.000000,5000.000000


In [15]:
#display advanced summary and export it to html file
Landslide_basics
Landslide_basics.to_file("Landslide_Summary.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Summary Report
### Baseline summary depicts further support that no missing data is present, the variables/features are set up properly, and significant statistics are present.
### Data shows emergence of data collection around 2008 and sporadic changes with slight growth in the years ahead.
### Location Accuracy 